Lab 4: Extending Logistic Regression

Cameron Matson

Zihao Mao

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
 
import os

In [2]:
# first lests load the datasets in

data_path = '../data/basketball'
players = pd.read_csv(os.path.join(data_path, 'Players.csv'))
players.head()

,Unnamed: 0,Player,height,weight,collage,born,birth_city,birth_state
0,0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


We probably don't need the "collage [sic]" or the their birth locationg.  Probably don't really need their birth year either, but it might be interesting to look at generational splits.  Also that unnamed column looks just like the index, so we can drop that too.

In [3]:
players.drop(['Unnamed: 0', 'collage', 'birth_city', 'birth_state'], axis=1, inplace=True)
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3922 entries, 0 to 3921
Data columns (total 4 columns):
Player    3921 non-null object
height    3921 non-null float64
weight    3921 non-null float64
born      3921 non-null float64
dtypes: float64(3), object(1)
memory usage: 122.6+ KB


Good.  They're all non null, and seem to be the correct datatype.

Now let's load the players stats.

In [4]:
stats = pd.read_csv(os.path.join(data_path, 'seasons_stats.csv'))
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24691 entries, 0 to 24690
Data columns (total 53 columns):
Unnamed: 0    24691 non-null int64
Year          24624 non-null float64
Player        24624 non-null object
Pos           24624 non-null object
Age           24616 non-null float64
Tm            24624 non-null object
G             24624 non-null float64
GS            18233 non-null float64
MP            24138 non-null float64
PER           24101 non-null float64
TS%           24538 non-null float64
3PAr          18839 non-null float64
FTr           24525 non-null float64
ORB%          20792 non-null float64
DRB%          20792 non-null float64
TRB%          21571 non-null float64
AST%          22555 non-null float64
STL%          20792 non-null float64
BLK%          20792 non-null float64
TOV%          19582 non-null float64
USG%          19640 non-null float64
blanl         0 non-null float64
OWS           24585 non-null float64
DWS           24585 non-null float64
WS          

There are a lot of fields here, and they're pretty inconsistently filled.  Some of this arises from the fact that its such a long timeline.  For example, in 1950, there was no such thing as a 3-pointer, so it wouldn't make sense for those players to have 3pt% stats. 

Inspecting the dataset a little further, we notice that there is no stat for points per game (PPG).  The total number of points scored is listed, but that is hard to compare across seasons where they played different games.  To make the dataset more valid, i.e. to make the points column a valid comparisson measure, we'll only consider seasons in which they played the current full 82 game schedule.  Which doesn't reduce the power of the dataset by that much, they moved to a 82 game season in 1967, and only the lockout shortened 1998-99 season didn't have a full scehdule.

Actually we might want to limit it to just seasons after 1980 when they introduced the 3 pointer.  That should just make the prediction task easier, although we lose even more of the dataset.  But if we consider the business case as being how to decide players posisitions *TODAY* it makes sense.

In [5]:
stats = stats[stats.Year >= 1980]
stats = stats[stats.Year != 1998]
stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18380 entries, 5727 to 24690
Data columns (total 53 columns):
Unnamed: 0    18380 non-null int64
Year          18380 non-null float64
Player        18380 non-null object
Pos           18380 non-null object
Age           18380 non-null float64
Tm            18380 non-null object
G             18380 non-null float64
GS            17686 non-null float64
MP            18380 non-null float64
PER           18375 non-null float64
TS%           18307 non-null float64
3PAr          18295 non-null float64
FTr           18295 non-null float64
ORB%          18375 non-null float64
DRB%          18375 non-null float64
TRB%          18375 non-null float64
AST%          18375 non-null float64
STL%          18375 non-null float64
BLK%          18375 non-null float64
TOV%          18321 non-null float64
USG%          18375 non-null float64
blanl         0 non-null float64
OWS           18380 non-null float64
DWS           18380 non-null float64
WS       

Now lets just focus on a few categories
- Player
- Year
- Age
- games played (G)
- minutes played (MP)
- field goals, field goal attempts, and percentage (FG, FGA, FG%)
- free throws (FT, FTA, FT%), two-pointers (2P, 2PA, 2P%), and three-pointers (3P, 3PA, 3P%)
- offensive, defensive, and total rebounds (ORB, DRB, TRB)
- assists (AST)
- steals (STL)
- blocks (BLK)
- turnovers (TOV)
- personal fouls (PF)
- points (PTS)

And of course our label: position.  We could probably use any of the features as a label actually, and see if one could predict performance in one aspect of the game based on info in the another.  But for now we'll stick with predicting position.


In [6]:
stats_to_keep = {'Player', 'Year','Pos', 'Age', 'G', 'MP', 'FG', 'FGA', 'FG%', 'FT', 'FTA', 'FT%',
                '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
                'TOV', 'PF', 'PTS'}

stats_to_drop = set(stats.columns)-stats_to_keep
stats.drop(stats_to_drop, axis=1, inplace=True)
stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18380 entries, 5727 to 24690
Data columns (total 27 columns):
Year      18380 non-null float64
Player    18380 non-null object
Pos       18380 non-null object
Age       18380 non-null float64
G         18380 non-null float64
MP        18380 non-null float64
FG        18380 non-null float64
FGA       18380 non-null float64
FG%       18295 non-null float64
3P        18380 non-null float64
3PA       18380 non-null float64
3P%       14969 non-null float64
2P        18380 non-null float64
2PA       18380 non-null float64
2P%       18266 non-null float64
FT        18380 non-null float64
FTA       18380 non-null float64
FT%       17657 non-null float64
ORB       18380 non-null float64
DRB       18380 non-null float64
TRB       18380 non-null float64
AST       18380 non-null float64
STL       18380 non-null float64
BLK       18380 non-null float64
TOV       18380 non-null float64
PF        18380 non-null float64
PTS       18380 non-null float64

To take care of some of the null values, when players had 0 attempts in a shooting category (FG, 3P, 2P, FT) they left the percentage field blank (can't divide by 0), but for our purposes its probably okay if we just say it was 0%.

In [7]:
stats['3P%'] = stats['3P%'].fillna(0)
stats['2P%'] = stats['2P%'].fillna(0)
stats['FT%'] = stats['FT%'].fillna(0)
stats['FG%'] = stats['FG%'].fillna(0)


Okay.  Finally, let's add the player description data to the stats dataframe.

In [8]:
stats['height'] = np.nan
stats['weight'] = np.nan
stats['born'] = np.nan

In [9]:
iplayer = players.set_index(keys='Player')
istats = stats.reset_index(drop=True)
for i, row in istats.iterrows():
    name = row[1]
    h = iplayer.loc[name].loc['height']
    w = iplayer.loc[name].loc['weight']
    b = iplayer.loc[name].loc['born']
    istats.iloc[i, 27] = h
    istats.iloc[i, 28] = w
    istats.iloc[i, 29] = b

In [10]:
stats = istats
stats.info()
stats.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18380 entries, 0 to 18379
Data columns (total 30 columns):
Year      18380 non-null float64
Player    18380 non-null object
Pos       18380 non-null object
Age       18380 non-null float64
G         18380 non-null float64
MP        18380 non-null float64
FG        18380 non-null float64
FGA       18380 non-null float64
FG%       18380 non-null float64
3P        18380 non-null float64
3PA       18380 non-null float64
3P%       18380 non-null float64
2P        18380 non-null float64
2PA       18380 non-null float64
2P%       18380 non-null float64
FT        18380 non-null float64
FTA       18380 non-null float64
FT%       18380 non-null float64
ORB       18380 non-null float64
DRB       18380 non-null float64
TRB       18380 non-null float64
AST       18380 non-null float64
STL       18380 non-null float64
BLK       18380 non-null float64
TOV       18380 non-null float64
PF        18380 non-null float64
PTS       18380 non-null float64
he

,Year,Player,Pos,Age,G,MP,FG,FGA,FG%,3P,...,TRB,AST,STL,BLK,TOV,PF,PTS,height,weight,born
0,1980.0,Kareem Abdul-Jabbar*,C,32.0,82.0,3143.0,835.0,1383.0,0.604,0.0,...,886.0,371.0,81.0,280.0,297.0,216.0,2034.0,218.0,102.0,1947.0
1,1980.0,Tom Abernethy,PF,25.0,67.0,1222.0,153.0,318.0,0.481,0.0,...,191.0,87.0,35.0,12.0,39.0,118.0,362.0,201.0,99.0,1954.0
2,1980.0,Alvan Adams,C,25.0,75.0,2168.0,465.0,875.0,0.531,0.0,...,609.0,322.0,108.0,55.0,218.0,237.0,1118.0,206.0,95.0,1954.0
3,1980.0,Tiny Archibald*,PG,31.0,80.0,2864.0,383.0,794.0,0.482,4.0,...,197.0,671.0,106.0,10.0,242.0,218.0,1131.0,185.0,68.0,1948.0
4,1980.0,Dennis Awtrey,C,31.0,26.0,560.0,27.0,60.0,0.450,0.0,...,115.0,40.0,12.0,15.0,27.0,66.0,86.0,208.0,106.0,1948.0


In [11]:
stats.drop(['Year','Player','born'], axis=1, inplace=True)
positions = stats['Pos'] # restore the position data for testing classifications
stat = stats
stat.info()
stat.drop(['Pos'], axis=1, inplace=True) #remove the position feature from the data set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18380 entries, 0 to 18379
Data columns (total 27 columns):
Pos       18380 non-null object
Age       18380 non-null float64
G         18380 non-null float64
MP        18380 non-null float64
FG        18380 non-null float64
FGA       18380 non-null float64
FG%       18380 non-null float64
3P        18380 non-null float64
3PA       18380 non-null float64
3P%       18380 non-null float64
2P        18380 non-null float64
2PA       18380 non-null float64
2P%       18380 non-null float64
FT        18380 non-null float64
FTA       18380 non-null float64
FT%       18380 non-null float64
ORB       18380 non-null float64
DRB       18380 non-null float64
TRB       18380 non-null float64
AST       18380 non-null float64
STL       18380 non-null float64
BLK       18380 non-null float64
TOV       18380 non-null float64
PF        18380 non-null float64
PTS       18380 non-null float64
height    18380 non-null float64
weight    18380 non-null float64
d

In [12]:
#normalize the numeric datas
from sklearn import preprocessing
X = stat.values
y = stat.columns
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X)
stat = pd.DataFrame(x_scaled, columns = y)
stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18380 entries, 0 to 18379
Data columns (total 26 columns):
Age       18380 non-null float64
G         18380 non-null float64
MP        18380 non-null float64
FG        18380 non-null float64
FGA       18380 non-null float64
FG%       18380 non-null float64
3P        18380 non-null float64
3PA       18380 non-null float64
3P%       18380 non-null float64
2P        18380 non-null float64
2PA       18380 non-null float64
2P%       18380 non-null float64
FT        18380 non-null float64
FTA       18380 non-null float64
FT%       18380 non-null float64
ORB       18380 non-null float64
DRB       18380 non-null float64
TRB       18380 non-null float64
AST       18380 non-null float64
STL       18380 non-null float64
BLK       18380 non-null float64
TOV       18380 non-null float64
PF        18380 non-null float64
PTS       18380 non-null float64
height    18380 non-null float64
weight    18380 non-null float64
dtypes: float64(26)
memory usage:

# PCA

Use PCA to reduce the demension of the data to 2 features

In [13]:
from sklearn.decomposition import PCA
n_components = 2
print ("Extracting the top %d eigenfaces from %d faces" % (
    n_components, stats.shape[1]))
pca = PCA(n_components=n_components)
%time 
stat_pca = pca.fit(stat.copy()).transform(stat.copy())

print ('pca:', pca.components_)

Extracting the top 2 eigenfaces from 26 faces
Wall time: 0 ns
pca: [[ 0.00340159  0.44303303  0.40148528  0.24562555  0.24652775  0.0575184
   0.06574366  0.08350179  0.06568503  0.22400359  0.22044924  0.05694294
   0.17577517  0.19441659  0.12215646  0.13198765  0.18670172  0.17231213
   0.13069441  0.16437958  0.06911151  0.20788636  0.29651674  0.23421354
   0.00400838  0.00074279]
 [-0.0575088  -0.03130873 -0.03561589 -0.01079286 -0.05525003  0.07681439
  -0.20787281 -0.2522501  -0.52164214  0.06603528  0.04409344  0.04931209
   0.01162407  0.05261553 -0.3721118   0.22906628  0.18580128  0.20807638
  -0.15856858 -0.09161807  0.1527345  -0.01743282  0.16505092 -0.03208916
   0.37121133  0.32060454]]


In [14]:
stat = pd.DataFrame(stat_pca, columns = ['first', 'second'])

In [15]:
stat.head()


,first,second
0,1.625438,0.553955
1,0.013658,0.153532
2,0.923291,0.292951
3,0.966552,-0.279464
4,-0.512837,0.189599


# Logic Regressions

In [16]:
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    

In [17]:
# inherit from base class
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    #private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    def _get_gradient(self,X,y):
        # programming \sum_i (yi-g(xi))xi
        gradient = np.zeros(self.w_.shape) # set gradient to zero
        for (xi,yi) in zip(X,y):
            # the actual update inside of sum
            gradi = (yi - self.predict_proba(xi,add_bias=False))*xi 
            # reshape to be column vector and add to gradient
            gradient += gradi.reshape(self.w_.shape) 
        
        return gradient/float(len(y))
       
    # public:
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 

            
blr = BinaryLogisticRegression(0.1)
print(blr)

Untrained Binary Logistic Regression Object


In [18]:
from scipy.optimize import minimize_scalar
import copy
class LineSearchLogisticRegression(BinaryLogisticRegression):
    
    # define custom line search for problem
    @staticmethod
    def line_search_function(eta,X,y,w,grad,C):
        wnew = w + grad*eta
        yhat = (1/(1+np.exp(-X @ wnew)))>0.5
        return np.sum((y-yhat)**2)-C*np.sum(wnew**2)
        
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            
            # do line search in gradient direction, using scipy function
            opts = {'maxiter':self.iters/20} # unclear exactly what this should be
            res = minimize_scalar(self.line_search_function, # objective function to optimize
                                  bounds=(self.eta/1000,self.eta*10), #bounds to optimize
                                  args=(Xb,y,self.w_,gradient,self.C), # additional argument for objective function
                                  method='bounded', # bounded optimization for speed
                                  options=opts) # set max iterations
            
            eta = res.x # get optimal learning rate
            self.w_ += gradient*eta # set new function values
                
            


In [19]:
from scipy.special import expit

class VectorBinaryLogisticRegression(BinaryLogisticRegression):
    # inherit from our previous class to get same functionality
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # but overwrite the gradient calculation
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        return gradient.reshape(self.w_.shape)


In [20]:
class LogisticRegression:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y==yval # create a binary problem
            # train the binary classifier for this class
            blr = VectorBinaryLogisticRegression(self.eta,self.iters)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row

In [21]:
class RegularizedBinaryLogisticRegression(VectorBinaryLogisticRegression):
    # extend init functions
    def __init__(self, C=0.0, **kwds):        
        # need to add to the original initializer 
        self.C = C
        # but keep other keywords
        super().__init__(**kwds) # call parent initializer
        
        
    # extend previous class to change functionality
    def _get_gradient(self,X,y):
        # call get gradient from previous class
        gradient = super()._get_gradient(X,y)
        
        # add in regularization (to all except bias term)
        gradient[1:] += -2 * self.w_[1:] * self.C
        return gradient
        

In [22]:
# now redefine the Logistic Regression Function where needed
class RegularizedLogisticRegression(LogisticRegression):
    def __init__(self, C=0.0, **kwds):        
        # need to add to the original initializer 
        self.C = C
        # but keep other keywords
        super().__init__(**kwds) # call parent initializer
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y==yval # create a binary problem
            # train the binary classifier for this class
            blr = RegularizedBinaryLogisticRegression(eta=self.eta,
                                                      iterations=self.iters,
                                                      C=self.C)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

# Split the data for test 

In [23]:
from sklearn.model_selection import ShuffleSplit

if 'Pos' in stat:
    positions = df_imputed['Survived'].values 
    del df_imputed['Survived'] 
    norm_features = ['Age','Fare' ]
    df_imputed[norm_features] = (df_imputed[norm_features]-df_imputed[norm_features].mean()) / df_imputed[norm_features].std()
    X = df_imputed.values 


num_cv_iterations = 3
num_instances = len(positions)
cv_object = ShuffleSplit(
                         n_splits=num_cv_iterations,
                         test_size  = 0.2)
print(cv_object)

ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)


In [24]:
X = 
print(positions.shape)
print(stat.values.shape)

SyntaxError: invalid syntax (<ipython-input-24-ba0ca3630f7a>, line 1)

In [ ]:
from sklearn import metrics as mt
from sklearn.metrics import accuracy_score

%time
lr = LogisticRegression(0.1,500)
lr.fit(X,y)

yhat = lr.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

In [ ]:
iter_num=0

for train_indices, test_indices in cv_object.split(X,y): 

    
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    # train the reusable logisitc regression model on the training data
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    iter_num+=1
    